<a href="https://colab.research.google.com/github/turnleftorgo/Artificial-Neural-Network/blob/main/logistic%20regression%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import necessary libraries
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler  # for feature scaling
from sklearn.model_selection import train_test_split  # for train/test split

"""
torch is Pytorch : A powerful and flexible deep learning framework that allows you to build and train neural networks.

torch.nn : Pytorch Neural network module

NumPy: A fundamental package for scientific computing in Python. like Array Operations , Provides support for large,
multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these arrays.

StandardScaler ensures that the features have similar scales, which is important for algorithms that are sensitive to the scales of input data,
such as gradient descent-based methods.

sklearn : a open-source machine learning library
"""

# Prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape
print(f'number of samples: {n_samples}, number of features: {n_features}')

# split data to 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# scale data
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

"""
Many machine learning algorithms perform better when features are on a similar scale.
This is especially true for algorithms that rely on gradient descent, like neural networks.
"""

# convert to tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.int64))
y_test = torch.from_numpy(y_test.astype(np.int64))

"""
PyTorch uses tensors as its fundamental data structure, similar to how NumPy uses arrays.
Tensors are required for creating and training neural networks in PyTorch.
Tensors are similar to NumPy arrays but come with additional functionalities like GPU support and automatic differentiation,
which are crucial for deep learning.
"""

"""Ensures that the data type of X_train is float32. Converts the NumPy array to a PyTorch tensor."""

"""classification targets are usually represented as integers, and PyTorch's loss functions expect the target labels to be in this format."""

# Create model
# f = wx + b, softmax at the end
class LogisticRegression(nn.Module):

    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 2)

    def forward(self, x):
        logits = self.linear(x)
        pred_probs = torch.nn.Softmax(dim=-1)(logits) #for asking question only
        return logits #return the logits

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LogisticRegression(n_features).to(device)  #load the model to the current device

# Loss and optimizer
learning_rate = 0.01
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# training loop
num_epochs = 200

for epoch in range(num_epochs):
    # forward pass and loss
    X_train, y_train = X_train.to(device), y_train.to(device) #load the data to device (GPU or CPU)
    logits = model(X_train)

    loss = loss_fn(logits, y_train.squeeze().long())

    # backward pass to compute the gradient
    loss.backward()

    # updates the model parameter based on the gradient
    optimizer.step()

    # zero gradients
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

'2.3.1+cu121'